In [1]:
import numpy as np
from tqdm import tqdm
from ecit import *

/opt/anaconda3/envs/graph/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.random.seed(1)
random.seed(1)

t = 1000
n_list = [800, 1200, 1600]
ensCIT = [(cmiknn, 400, p_alpha05)]
zDiss = ['gaussian'] 
noiseDiss = ['t4','t3','t2','t1'] 

results_table = []

for n in n_list:
    for zDis in zDiss:
        for cit, k, p_ensemble in ensCIT:
            row = [n, zDis.capitalize(), cit.__name__ + str(k) + p_ensemble.__name__]

            for noiseDis in tqdm(noiseDiss, desc=f"{n:>5}, {zDis:>8}, {k:>3},{p_ensemble.__name__:>11}"):
                eI = 0
                eII = 0
                for i in range(t):
                    retries = 0
                    while retries < 5:
                        try:
                            dataI = np.hstack((generate_samples(n=n,indp='C',z_dis=zDis, noise_dis=noiseDis,noise_std=1)))
                            dataII = np.hstack((generate_samples(n=n,indp='N',z_dis=zDis, noise_dis=noiseDis,noise_std=1)))
                            obj_ECIT = ECIT(dataI, cit, p_ensemble, k if k<100 else int(n/k))
                            pI = obj_ECIT([0], [1], [2])
                            obj_ECIT = ECIT(dataII, cit, p_ensemble, k if k<100 else int(n/k))
                            pII = obj_ECIT([0], [1], [2])
                            break
                        except Exception as e:
                            retries += 1
                            print(f"Retry times {retries}: {e}")
                            if retries > 5: raise e
                    if pI<0.05:
                        eI += 1
                    if pII>0.05:
                        eII += 1
                eI = eI/t
                power = (t - eII)/t
                row.append(eI)
                row.append(power)
            results_table.append(row)

results_table

 1200, gaussian, 400,  p_alpha05:   0%|          | 0/4 [00:00<?, ?it/s]/opt/anaconda3/envs/graph/lib/python3.10/site-packages/numpy/_core/_methods.py:127: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
 1200, gaussian, 400,  p_alpha05:  25%|██▌       | 1/4 [6:47:56<20:23:48, 24476.08s/it]/opt/anaconda3/envs/graph/lib/python3.10/site-packages/numpy/_core/_methods.py:127: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
 1200, gaussian, 400,  p_alpha05:  50%|█████     | 2/4 [13:36:05<13:36:07, 24483.96s/it]/opt/anaconda3/envs/graph/lib/python3.10/site-packages/numpy/_core/_methods.py:127: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/opt/anaconda3/envs/graph/lib/python3.10/site-packages/numpy/_core/_methods.py:127: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axi

[[800,
  'Gaussian',
  'cmiknn400p_alpha05',
  0.134,
  0.966,
  0.125,
  0.969,
  0.123,
  0.952,
  0.137,
  0.916],
 [1200,
  'Gaussian',
  'cmiknn400p_alpha05',
  0.133,
  0.988,
  0.159,
  0.983,
  0.141,
  0.973,
  0.135,
  0.951],
 [1600,
  'Gaussian',
  'cmiknn400p_alpha05',
  0.172,
  0.992,
  0.176,
  0.986,
  0.153,
  0.986,
  0.137,
  0.968]]

In [3]:
data = results_table

data = [
    [round(item, 3) if isinstance(item, float) else item for item in sublist]
    for sublist in data
]
for row in data:
    print("{:<6} {:<12} {:<18} {:<6.3f} {:<6.3f} {:<6.3f} {:<6.3f} {:<6.3f} {:<6.3f}".format(*row))

800    Gaussian     cmiknn400p_alpha05 0.134  0.966  0.125  0.969  0.123  0.952 
1200   Gaussian     cmiknn400p_alpha05 0.133  0.988  0.159  0.983  0.141  0.973 
1600   Gaussian     cmiknn400p_alpha05 0.172  0.992  0.176  0.986  0.153  0.986 
